In [49]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key  = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")

In [50]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load()
docs


[Document(metadata={'source': 'apjspeech.pdf', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowin

In [51]:
from langchain import PromptTemplate

template = """ Write a concise and hort summary of the following speech,
Speech: {text}


"""

prompt = PromptTemplate(input_variable="text", template=template)


In [52]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm,chain_type='stuff',prompt=prompt,verbose=True)


In [53]:
chain.run(docs)
output_summary = chain.run(docs)
output_summary



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Write a concise and hort summary of the following speech,
Speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online inte

'In his departing speech, President A.P.J. Abdul Kalam reflects on his five years in office, highlighting key takeaways from his interactions with diverse sectors of Indian society. \n\nHe emphasizes the aspirations of India\'s youth to build a developed nation by 2020, urging the empowerment of villages and rural areas. \n\nKalam stresses the importance of leveraging rural core competence for competitive advantage and promoting agricultural growth through the "Seed to Food" principle.  He underscores the courage and resilience of Indians in overcoming challenges, exemplified by disaster relief efforts and the spirit of innovation.\n\nHe champions connectivity as a tool for societal transformation, citing the Pan African e-Network as an example.  Kalam pays tribute to India\'s defense forces and para-military personnel for their dedication.\n\nFinally, he encourages the youth to embrace value-based education and leadership, envisioning a developed India characterized by reduced rural-u

In [59]:
### Map reduce to summarize large documents
from langchain.text_splitter import RecursiveCharacterTextSplitter

final_docs = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100).split_documents(docs)
final_docs


[Document(metadata={'source': 'apjspeech.pdf', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology,'),
 Document(metadata={'source': 'apjspeech.pdf', 'page': 0}, page_content='of you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi 

In [55]:

from langchain import PromptTemplate
chunks_prompt="""
Please summarize the below speech:
Speech:`{text}'
Summary:
"""
map_prompt = PromptTemplate(input_variable="text", template=chunks_prompt)


In [56]:
# summary_chain = load_summarize_chain(
#     llm=llm,
#     chain_type='map_reduce',
#     map_prompt=map_prompt
# )

In [57]:
final_prompt = """
Provide the final summary of the enture speech with these important points.
Add a motication title to the summary.Start the precise summary with an introduction and
provide the summary in number points for the speech {text}
"""
final_template_prompt = PromptTemplate(input_variable="text", template=final_prompt)    


In [61]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=final_template_prompt,
    verbose=True
)

In [62]:
output = summary_chain.run(final_docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech:`A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology,'
Summary:

Prompt after formatting:

Please summarize the below speech:
Speech:`of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future weal

c:\Users\moham\Desktop\Generative AI\generative-ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (5305 > 1024). Running this sequence through the model will result in indexing errors




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Provide the final summary of the enture speech with these important points.
Add a motication title to the summary.Start the precise summary with an introduction and
provide the summary in number points for the speech In his departing speech, A.P.J. Abdul Kalam expresses his joy at addressing the nation and those abroad after completing five fulfilling years as President. He expresses gratitude for the enriching experience of working with people from diverse backgrounds in politics, science, and technology, emphasizing the positive impact of their association.  


The speaker is addressing a diverse group of people from various sectors of society, including politics, science, academia, arts, business, judiciary, administration, local bodies, farming, homemakers, the media, and especially the youth and student community. 

They are emphasizing the importance of this wide representat

In [63]:
output

'##  A Vision of Progress: Kalam\'s Call for a United and Prosperous India\n\nThis powerful speech by A.P.J. Abdul Kalam outlines a clear roadmap for India\'s future, emphasizing youth empowerment, rural development, and agricultural growth as key pillars of national progress. \n\n**Key Points:**\n\n1. **Youth as Agents of Change:**  Kalam recognizes the aspirations of India\'s youth, viewing them as the driving force behind the nation\'s development by 2020.\n2. **Empowering Rural India:**  He stresses the importance of transforming India\'s villages through local development initiatives, strong urban-rural connections, and the Periyar PURA model, which focuses on sustainable agriculture, renewable energy, and local industries.\n3. **Agriculture: The Foundation for Growth:** Kalam champions the "Seed to Food" principle, advocating for increased agricultural production and a collaborative approach involving farmers, scientists, and industry. \n4. **Resilience and Inspiration:** Drawing